In [24]:
!pip install geopandas -qq
!pip install pandas_geojson -qq
import panel as pn
pn.extension(sizing_mode="stretch_width")
import geopandas as gpd
import pandas as pd
import folium as fm
import matplotlib.pyplot as plt
import numpy as np
import json
import urllib.request

# Opening JSON file
#f = open('https://raw.githubusercontent.com/dosm-malaysia/data-open/main/datasets/geodata/electoral_1_dun.geojson')
url = 'https://raw.githubusercontent.com/dosm-malaysia/data-open/main/datasets/geodata/electoral_1_dun.geojson'

geo_json = json.load(urllib.request.urlopen(url))

duns = gpd.read_file("https://raw.githubusercontent.com/dosm-malaysia/data-open/main/datasets/geodata/electoral_1_dun.geojson")

sociodemo = pd.read_csv('https://github.com/booluckgmie/podac/raw/main/sociodemo.csv')

#READ DATAFRAME
from pandas_geojson import read_geojson, filter_geojson
filter = ['Pulau Pinang']
select = filter_geojson(geo_json=geo_json, filter_list=filter, property_key='state')

#CREATE SELECTION
select = pn.widgets.Select(name='Select DUN', options=["N.01 Penaga",
"N.02 Bertam",
"N.03 Pinang Tunggal",
"N.04 Permatang Berangan",
"N.05 Sungai Dua",
"N.06 Telok Ayer Tawar",
"N.07 Sungai Puyu",
"N.08 Bagan Jermal",
"N.09 Bagan Dalam",
"N.10 Seberang Jaya",
"N.11 Permatang Pasir",
"N.12 Penanti",
"N.13 Berapit",
"N.14 Machang Bubuk",
"N.15 Padang Lalang",
"N.16 Perai",
"N.17 Bukit Tengah",
"N.18 Bukit Tambun",
"N.19 Jawi",
"N.20 Sungai Bakap",
"N.21 Sungai Acheh",
"N.22 Tanjong Bunga",
"N.23 Air Putih",
"N.24 Kebun Bunga",
"N.25 Pulau Tikus",
"N.26 Padang Kota",
"N.27 Pengkalan Kota",
"N.28 Komtar",
"N.29 Datok Keramat",
"N.30 Sungai Pinang",
"N.31 Batu Lancang",
"N.32 Seri Delima",
"N.33 Air Itam",
"N.34 Paya Terubong",
"N.35 Batu Uban",
"N.36 Pantai Jerejak",
"N.37 Batu Maung",
"N.38 Bayan Lepas",
"N.39 Pulau Betong",
"N.40 Telok Bahang"])

select
#select_option = pn.Row(select.controls(jslink=True), select)

Select(options=['N.01 Penaga', ...], sizing_mode='stretch_width', value='N.01 Penaga')

In [25]:
selectv = select.value

from pandas_geojson import read_geojson, filter_geojson
filter = [selectv]
selectgeo = filter_geojson(geo_json=geo_json, filter_list=filter, property_key='dun')

import folium as fm
from folium import plugins

map = fm.Map(location = [5.016806622688496, 107.9456777002687], tiles='cartodbpositron', zoom_start = 6)

# Adds tool measure to the top right
#from folium.plugins import MeasureControl
#map.add_child(MeasureControl())

#map geojson
fm.GeoJson(selectgeo, name='DUN boundary',tooltip=fm.GeoJsonTooltip(fields=['dun', 'parlimen'])).add_to(map)
map.fit_bounds(map.get_bounds(), padding=(30, 30))
#map
folium_map = pn.panel(map, height=400)
#folium_map

select_sociodemo = sociodemo[sociodemo['dun']== selectv]
select_sociodemo['watercover'] = np.round(select_sociodemo.watercover, decimals = 3)
max_elev = select_sociodemo['max_elevation'].iloc[0]
water = select_sociodemo['watercover'].iloc[0]
pop = select_sociodemo['population_total'].iloc[0]

number = pn.indicators.Number(format='{value}'    
)

indicator = pn.Row(number.clone(name='Population', value=pop, colors=[(33, 'black')]), 
       number.clone(name='Maximum Elevation', value=max_elev, colors=[(66, 'blue')]), 
       number.clone(name='Surface covered in water', value=water, colors=[(100, 'red')]),sizing_mode='stretch_width')

#indicator
description = select_sociodemo['dun'].iloc[0]

#str_pane = pn.pane.Str(str(description), style={'font-size': '100pt'})
#str_pane

str_pane = pn.indicators.Number(name=str(description))

app2 = pn.Column(
    
    pn.Column(        
        str_pane,
        indicator,
        folium_map,
        sizing_mode='scale_both'
    ),
    sizing_mode='scale_both'
)
app2

<ipython-input-25-a1507b43a7d1>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_sociodemo['watercover'] = np.round(select_sociodemo.watercover, decimals = 3)


Column(sizing_mode='scale_both')
    [0] Column(sizing_mode='scale_both')
        [0] Number(name='N.10 Seberang Jaya')
        [1] Row(sizing_mode='stretch_width')
            [0] Number(colors=[(33, 'black')], name='Population', value=93836)
            [1] Number(colors=[(66, 'blue')], name='Maximum Elevation', value=38.0)
            [2] Number(colors=[(100, 'red')], name='Surface covered i..., value=0.123)
        [2] Folium(Map, height=400)

In [71]:
#app2.save('indicator.html', resources=INLINE, embed=True)

#app.save('datatable.html')